In [1]:
# import required packages
# import pyodbc
import pandas as pd
import numpy as np
from pandas.io import sql
# from tqdm import tqdm
import datetime

import sqlite3
sqlite_db = 'freight.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()
from IPython.display import clear_output

#### Add records to the tables
``` python
df.to_sql('Name_of_table',
            con=conn,
            if_exists='replace',
            index=False)
```

#### Read records from the table
``` python
sql.read_sql('select * from Name_of_table limit 10', con=conn)
```

In [2]:
# load the library
import requests
import bs4
from bs4 import BeautifulSoup
import urllib, requests
import pandas as pd
from tqdm import tqdm
import sys
sys.setrecursionlimit(10000)

In [3]:
def parse_table(table):
    """ Get data from table """
    return [
        [cell.get_text().strip() for cell in row.find_all(['th', 'td'])]
           for row in table.find_all('tr')
    ]

In [17]:
def get_html(num):
    URLa = 'https://www.bigschedules.com/indexSlide/home-bigschedules'
    URLb = '.html'
    URL = URLa+str(num)+URLb
    html_content = BeautifulSoup(urllib.request.urlopen(URL), "lxml")
    return html_content

In [5]:
def get_coord(html_content):
    tmp_coord = html_content.find('a', attrs={'class': 'various'}).attrs['href']
    for i in range(len(tmp_coord)):
        if tmp_coord[i:i+2] == "y=":
            pt_a = i
        if tmp_coord[i:i+2] == "x=":
            pt_b = i
    Latitude = tmp_coord[pt_a+2:pt_b-1]
    Longitude = tmp_coord[pt_b+2:]
    return Latitude, Longitude

In [6]:
def tabulate_html(html_content, num):
    web_table = html_content.find('table', attrs={'class': 'de_box_table'})
    pd_tbl_orig = parse_table(web_table)
    pd_tbl = pd.DataFrame(pd_tbl_orig[1:])
    pd_tbl = pd_tbl.transpose()
    tmp_header = pd_tbl.iloc[0:1]
    tmp_data = pd_tbl.iloc[1:2]
    tmp_df = pd.DataFrame()
    tmp_df = tmp_df.append(tmp_data)
    tmp_df.columns = tmp_header.transpose()[0]
    y, x = get_coord(html_content)
    tmp_df['Latitude'] = y
    tmp_df['Longitude'] = x
    tmp_df['ID'] = num
    return tmp_df

In [7]:
def build_tbl(start, end, first=False):
    if first:
        master_df = pd.DataFrame()
    else:
        try:
            master_df = sql.read_sql('select * from housing', con=conn)
        except:
            master_df = pd.DataFrame()
    for i in tqdm(range(start,end)):
        html_content = get_html(i)
        success = False
        try:
            page_df = tabulate_html(html_content, i)
            success = True
        except:
            pass
        if success:
            if len(master_df) <= 0:
                master_df = master_df.append(page_df)
            elif list(page_df['ID']) not in list(master_df['ID']) :
                master_df = master_df.append(page_df)
            
            try:
                page_df.to_sql('housing', con=conn, if_exists='append', index=False)
            except:
                master_df.to_sql('housing', con=conn, if_exists='replace', index=False)
            progress = pd.DataFrame([i], columns=['Count'])
            progress.to_sql('step', con=conn, if_exists='replace', index=False)
            clear_output()
    #save_data(master_df,'master_df')
    #save_data(i, 'step')
    return master_df

In [8]:
def reset_all_parameters():
    master_df = pd.DataFrame()
    save_data(master_df,'master_df')
    html_content = get_html(100)
    master_df = tabulate_html(html_content,100)
    save_data(master_df,'master_df')
    step = 0
    save_data(step,'step')
# reset_all_parameters()

In [18]:
# 124700 - 235270
# 235271 - 360000

In [13]:
progress = pd.DataFrame([167389], columns=['Count'])
progress.to_sql('step', con=conn, if_exists='replace', index=False)

In [6]:
start = sql.read_sql('select * from step', con=conn)
start = start['Count'].iloc[0]
start = max(start, 124700)
# end = 365000
end = 235270
xtmp = build_tbl(start,end)

In [4]:
tmp = sql.read_sql('select * from housing', con=conn)
len(tmp)

62355

In [5]:
tmp.shape

(62355, 22)

In [43]:
tmp.columns

Index(['Status', 'Rent', 'Block and unit number', 'Floor', 'Room',
       'Gross area(sq feet)', 'Net floor area(sq feet)', 'Management Fee',
       'Property age(year)', 'Address', 'Views #', 'Bookmarked #',
       'Ads or renew date', 'Modified date', 'User last login', 'Expire date',
       'Latitude', 'Longitude', 'ID'],
      dtype='object')

In [47]:
tmp['Price']= np.NaN

In [2]:
sql.read_sql('select * from step', con=conn)

,Count
0,126703


In [3]:
sql.read_sql('select * from housing', con=conn)